## Sustitución categoria más frecuente  ==> Feature Engine


### Qué es Feature Engine?

Feature Engine es una librería de python que hemos creado para apoyar este curso. 

- Feature Engine incluye todas las técnicas de ingeniería de varaibles descritas en este curso
- Feature Engine funciona como Scikit-learn, por lo tanto es fácil de aprender
- Feature Engine te permite implementar pasos de ingeniería de variables espeficos para segmentos de datos específicos
- Feature Engine puede ser integrado con los flujos de trabajo de Scikit-learn pipeline permitiendo construir modelos facilmente
- 
**Feature Engine te permite diseñar y guardar un flujo de ingeneria de variables con procesos disenados especificamente para los diferentes grupos de variables.**

-------------------------------------------------------------------
Feature Engine puede ser instalado vía pip ==> pip install feature-engine

- Asegurate que haz instalado feature-engine antes de correr este notebook

Para más detalle visita nuestro website


## En este demo:

Vamos a usar **Feature Engine para hacer la sustitución por la categoriá más frecuente** usando los datos Ames House Price.

- Para bajar los datos, por favor referirse a la clase en **Datasets** en la  **Sección 1** del curso.

### Nota: 
* 'Imputer' se deriva del verbo en inglés 'to impute' que quiere decir sustituir o reemplazar. Imputer es el objeto que completa la sustitución, de ahi el nombre dado a la clase.




In [16]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# to split the datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#  feature engine
from feature_engine import missing_data_imputers as mdi

In [17]:
# carguemos los datos con unas columnas seleccionadas

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('../houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [18]:
data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
GarageYrBlt    0.055479
SalePrice      0.000000
dtype: float64

Todas las variables predictivas tienen datos ausentes

In [19]:
# separar datos en segmentos entrenamiento y prueba

# primero, separemos el target (SalePrice) del resto de las variables (features)

cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

## Feature Engine captura las variables categóricas automáticamente

In [20]:
# llamemos el imputer de feature engine
# no necesitamos especificar nada
imputer = mdi.FrequentCategoryImputer()

In [21]:
# ajustemos imputer

imputer.fit(X_train)

FrequentCategoryImputer(variables=['BsmtQual', 'FireplaceQu'])

In [22]:
# vemos que el imputer encontró las variables categóricas 
# para sustituir con la categoría mas frecuente o moda
imputer.variables

['BsmtQual', 'FireplaceQu']

In [23]:
# aqui vemos los valores  que serán usados
# para reemplazar los NA en cada variable
imputer.imputer_dict_

{'BsmtQual': 'TA', 'FireplaceQu': 'Gd'}

In [24]:
#revisemos los valores ajustados en el entrenamiento de prueba

X_train[imputer.variables].mode()

,BsmtQual,FireplaceQu
0,TA,Gd


In [25]:
# feature engine devuelve un dataframe

tmp = imputer.transform(X_train)
tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,Gd,NaN,573.0,1998.0
682,Gd,Gd,NaN,0.0,1996.0
960,TA,Gd,50.0,0.0,NaN
1384,TA,Gd,60.0,0.0,1939.0
1100,TA,Gd,60.0,0.0,1930.0


In [26]:
# revisemos que los valores nulos ya no existen

tmp[imputer.variables].isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
dtype: float64

## Feature engine te permite especificar grupos de variables fácilmente

In [27]:
# usemos la sustitución pero esta vez solo 
# usemos  2 de la 3 variables numéricas

imputer = mdi.FrequentCategoryImputer(variables=['BsmtQual'])

imputer.fit(X_train)

FrequentCategoryImputer(variables=['BsmtQual'])

In [28]:
# ahora el imputer solo tiene las variable que le indicamos

imputer.variables

['BsmtQual']

In [29]:
# y podemos ver el valor asignado para cada variable

imputer.imputer_dict_

{'BsmtQual': 'TA'}

In [30]:
# feature engine devuelve un dataframe
tmp = imputer.transform(X_train)


#revisemos que ya no tenemos valores nulos
tmp[imputer.variables].isnull().mean()

BsmtQual    0.0
dtype: float64

Funcionó!